# Information Visualization Altair Demo 3
School of Information, University of Michigan


## Plan
1. Review
2. Interaction 

## Resources/References 
*  [Interaction documentation](https://altair-viz.github.io/user_guide/interactions.html)
*  [UW course examples](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb)
*  [Multiple Interaction](https://altair-viz.github.io/gallery/multiple_interactions.html#gallery-multiple-interactions)






In [0]:
# imports we will use
import altair as alt
import pandas as pd
from vega_datasets import data as vega_data
car_url = vega_data.cars.url
cars = pd.read_json(car_url)

In [2]:
# see what's inside
cars.sample(3)

,Name,Miles_per_Gallon,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin
211,datsun b-210,32.0,4,85.0,70.0,1990,17.0,1976-01-01,Japan
168,chevroelt chevelle malibu,16.0,6,250.0,105.0,3897,18.5,1975-01-01,USA
393,datsun 310 gx,38.0,4,91.0,67.0,1995,16.2,1982-01-01,Japan


In [4]:
#0.1 exercise make a scatter plot with 
#   Horsepower(X) and Miles_per_Gallon(Y)
#   Encode Origin as color 
#   Name this chart hp_mpg 
hp_mpg=alt.Chart(cars).mark_circle(
    size=80,
    opacity=0.5
).encode(
    x="Horsepower:Q",
    y="Miles_per_Gallon:Q",
    color="Origin"
)
hp_mpg

alt.Chart(...)

## 1. Tooltip and default interactivity

We'll be talking about interactivity extensively in class but we'd like to introduce you to some of the Altair/Vega-Lite features now. One of the main philosophies in interactivity for infovis is "Overview first, zoom and filter, then details-on-demand." (called Shneiderman's mantra). It basically means that we should give the high level picture and support--through interaction--access to more.  

To that end: we're going to enable a couple of features in our plot. First, we want to know which car correspond to which dot. If we see an outlier or interesting point we'll want to find it. We'll do this by enabling [tooltips](https://altair-viz.github.io/gallery/scatter_tooltips.html).  Second, with complex data we want to be able to zoom in and move around so we can see more points. This is easily enabled in Altair with the `.interactive()`

In [7]:
#1.1 add tooltip and default interactivity 
hp_mpg.encode(
    tooltip=["Name:N","Origin:N"]
).interactive()

alt.Chart(...)

Note that there are other ways to simulate tooltips. For example, this [version](https://altair-viz.github.io/gallery/multiline_tooltip.html) achieves tooltips on the axes by having an invisible layer.

## 2 Selection
Altair handles interaction through [selection](https://altair-viz.github.io/user_guide/interactions.html#selections-building-blocks-of-interactions). An Altair *Selection* can 
1. handle input event 
2. determine whether or not a given data record lies within the selection 
3. (re-)configure the visualization based on the selection.



### Single Selections
To make a single mark selection (hovering, clicking, etc. on a single mark), you need:
1. make a single selection instance, called `alt.selection_single()`, which is bound to mouse click by default.
2. define what happens when the selection happens by adding a [condition](https://altair-viz.github.io/user_guide/generated/api/altair.condition.html#altair.condition) 
3. add this selection (STEP1) to your chart by calling `.add_selection()`
4. include your condition in `encode` or `transform_filter`

We will be using alt.condition a lot to specify what happens (step 2). Think of this as a fancy if-then-else. Usually we'll be doing things like this in the encoding:

`color = alt.condition(selection,"Origin",alt.value("gray")` which roughly means: if the thing is selected, keep it the original color, else set the the color to gray.

In [8]:
#2.1 : 4 steps for adding interactivity 

#step 1 create selection 
selection=alt.selection_single()
# selection.value
#step 2 -- when selected, use the original color, else make it gray
colorCondition=alt.condition(
    selection,
    "Origin",
    alt.value("gray")
)
hp_mpg.add_selection(
    #step3 add selection 
    selection
).encode(
    color=colorCondition
)

alt.Chart(...)

### 2.1 Selections are adjustable
Here are a list of things you can modify in a selection ([doc](https://altair-viz.github.io/user_guide/generated/core/altair.SingleSelection.html#altair.SingleSelection)). We will cover most of them today. 
* empty
* type 
* on
* nearest
* init
* bind
* clear
* encodings
* fields
* resolve


### 2.2 empty
By default, all data values are considered to be within an empty selection. When set to none, empty selections contain no data values.

In [11]:
#2.2.1 adjust size, and change default "empty"
                     
#step 1: uncomment this 
selection=alt.selection_single(
    empty="none"
);

# step 2 TODO: make a size condition 
sizeCondition=alt.condition(
    selection,
    alt.SizeValue(200),
    alt.SizeValue(50)
)
hp_mpg.add_selection(
    #step 3
    selection
).encode(
    #step 4,TODO: add size condition 
    size=sizeCondition
)


alt.Chart(...)

In [12]:
# 2.2.2 Three steps for making an interactive chart 
# put it all together

#1. define selection (nothing selected to start)
selection=alt.selection_single(empty="none");

#2. define condition
sizeCondition=alt.condition(selection,alt.SizeValue(200),alt.SizeValue(50))
colorCondition=alt.condition(selection,"Origin",alt.value("gray"))

#3. add selection and conditions to chart
hp_mpg.add_selection(
    selection
).encode(
    # 4. add conditions to encode: you can chain multiple selections 
    size=sizeCondition,
    color=colorCondition
)

alt.Chart(...)

###2.3 Type
There are 3 types of selections:
* **selection_single** - select a single discrete value, by default on click events.
* **selection_multi** - select multiple discrete values. The first value is selected on mouse click and additional values toggled using shift-click.
* **selection_interval** - select a continuous range of values, initiated by mouse drag.

In [14]:

# 2.3.1 multi

# here's an example of mutliple selection

#step1 TODO
selection=alt.selection_multi();

#step2
colorCondition=alt.condition(selection,"Origin",alt.value("gray"))

hp_mpg.add_selection(
    #step3
    selection
).encode(
    #step4
    color=colorCondition,
)

alt.Chart(...)

In [15]:
# 2.3.2 interval 

# example of an interval

# #step1, click drag to select a range in the x and y directions
selection=alt.selection_interval();

#step2
colorCondition=alt.condition(selection,"Origin",alt.value("gray"))

hp_mpg.add_selection(
    #step3
    selection
).encode(
    #step4
    color=colorCondition,
)

alt.Chart(...)

### 2.4 "On"

By default things work based on clicks, but you can use other event types like hovering.

A Vega [event stream](https://vega.github.io/vega/docs/event-streams/) (object or selector) triggers the selection. For interval selections, the event stream must specify a start and end.

You can customize using[ vega event selector syntax](https://vega.github.io/vega/docs/event-streams/) 

In [17]:

#2.4.1  mouseover instead of click 

#step1 
selection=alt.selection_single(
    on="mouseover"
)

#step2
colorCondition=alt.condition(selection,"Origin",alt.value("gray"))

hp_mpg.add_selection(
    #step3
    selection
).encode(
    #step4
    color=colorCondition,
)

alt.Chart(...)

### 2.5 Nearest 
When true, an invisible voronoi diagram is computed to accelerate discrete selection. The data value nearest the mouse cursor is added to the selection.


In [19]:

#2.5.1 
selection=alt.selection_single(on='mouseover',nearest=True)
colorCondition=alt.condition(selection,"Origin",alt.value("gray"))
hp_mpg.add_selection(selection).encode(
    color=colorCondition
)

alt.Chart(...)

### 2.6 Clear 
Clears the selection, emptying it of all values. Can be an EventStream or false to disable.

In [21]:

#2.6 clear

# select when mouseover, use the nearsest value, when the end-user clicks
# clear the selection
selection=alt.selection_single(on='mouseover', nearest=True,
                               clear="click")

colorCondition=alt.condition(selection,"Origin",alt.value("gray"))

hp_mpg.add_selection(selection).encode(
    color=colorCondition
)



alt.Chart(...)

### 2.7 [Init](https://vega.github.io/vega-lite/docs/init.html)

One trick you can use to focus attention is to initialize the value. Note that because selections work on x/y coordinates (where the mouse is) that's what we'll use. In this example we set a rectangle for the initial selection. 

In [23]:

#2.7.1
#with initial value  (we are using x, and y instead of HP and MPG)
selection=alt.selection_interval(
    init={
        "x":[60,150],
        "y":[15,30]
    }
)
colorCondition=alt.condition(selection,"Origin",alt.value("gray"))
hp_mpg.add_selection(selection).encode(
    color=colorCondition
)



alt.Chart(...)

### 2.8 encodings 

An array of encoding channels. The corresponding data field values must match for a data tuple to fall within the selection. A way to think of it is: what is the target of our selection? Is it an example of a "class" of points? Are we trying to select based on some feature (like the x value?)


In [27]:
#2.8.1
# # here, we're going to select other points based on the color of the selection
selection_single = alt.selection_single(
    encodings=["color"]
)

colorCondition1 = alt.condition(selection_single,"Origin",alt.value("gray"))

c1=hp_mpg.add_selection(selection_single).encode(
    color= colorCondition1
).properties()
c1

# # here, we're going to select other points based on the x value of what we clicked on

selection_interval=alt.selection_interval(
    encodings=["x"]
    )

colorCondition2 = alt.condition(selection_interval,"Origin",alt.value("gray"))

c2=hp_mpg.add_selection(selection_interval).encode(
    color = colorCondition2
).properties()
c2

c1|c2

alt.Chart(...)

In [0]:
# extra 
# using & | and ~ to do logical operations for selections 
# take a look at the color 
# make sure that you add individual selections to your chart.
s1=alt.selection_interval(encodings=["x"])
s2=alt.selection_interval(encodings=["y"])


c2=hp_mpg.add_selection(s1,s2).encode(
    color = alt.condition(s1&(~s2),"Origin",alt.value("gray"))
)

c3=hp_mpg.add_selection(s1,s2).encode(
    color = alt.condition(s1|s2,"Origin",alt.value("gray"))
)

c2|c3 

#### [Fields](alt.condition(selection_single,"Origin",alt.value("gray"))) 
Very similar to encodings.  Using field names instead of encoding schemas. 


### 2.9 Bind

**Short version**: Connect selection to scale or widgets

**Official definition**: Establish a two-way binding between a single selection and input elements (also known as dynamic query widgets). A binding takes the form of Vega’s input element binding definition or can be a mapping between projected field/encodings and binding definitions.




**Binding to "scales"** creates zooming and panning

One simple way to do this to say that we want to control the way zooming/panning works based on the "selection" (click and drag to see this)

In [29]:
#2.9.1 setting scales 

# has to be interval. We can "bind" the selection to 
# the axes instead of the data

selection=alt.selection_interval(
    bind="scales",
    encodings=["x"]
)

hp_mpg.add_selection(selection).encode(
    color="Origin:N"
).properties(title="zooming and panning")
# hp_mpg.interactive()

alt.Chart(...)

## 3. **Bind to  [interface widget](https://altair-viz.github.io/user_guide/interactions.html#input-element-binding)**

A more interesting objective is to bind the vis to widgets. Based on what happens in the widget, we'll change the data.

Widgets are built-in components that takes users'selection input. 
Altair has the following [widgets](https://altair-viz.github.io/gallery/multiple_interactions.html#gallery-multiple-interactions) 

1.   select (dropdown-menu)
2.   checkbox  (multiple selections)
3.   radio (single selection)
4.   range (value slider)



### 3.1 Drop-Down

Let's create a simple drop-down menu. We'll need to do the following:

1.   Obtain a list of options (to populate the drop-down)
2.   Create a selection, with field, init, and bind. ("bind" the widget)
3.   Create a condition
4.  Add condition and selection to your chart



In [31]:
#3.1.1 
# you can create the list of options any way you want
# here, we'll just use pandas to figure out what options we have

# step 1: Obtain a list of option 
origins=list(cars["Origin"].unique())
origins.sort()
origins

['Europe', 'Japan', 'USA']

In [32]:
#3.1.2 

# # step 2: create a selection 
selectOrigin=alt.selection_single(
    fields=["Origin"],#where our selection is based on 
    init={"Origin":origins[0]},
    bind=alt.binding_select(options=origins,name="Select Origin")
)

# #step 3
# # when selected, stick to the original color of the Origin
# # otherwise make it light gray
colorCondition = alt.condition(
    selectOrigin,
    alt.Color("Origin:N"),
    alt.value("gray")
    )

# #step 4 
hp_mpg.add_selection(
    selectOrigin
).encode(
    color=colorCondition
)

alt.Chart(...)

### 3.2 Radio button

Here's an example using radio buttons

In [33]:
# 3.2.1 radio, and name 
selectOrigin=alt.selection_single(
    fields=['Origin'],
    init={"Origin":origins[0]},
    # notice the binding_radio
    bind=alt.binding_radio(options=origins,name="Origin"),#edit this line
    name="Origin"
)
hp_mpg.add_selection(
    selectOrigin
).encode(
    color=alt.condition(selectOrigin,alt.Color("Origin:N"),alt.value('lightgray'))
)

alt.Chart(...)

### 3.3 Checkbox 
Unfortunately, there's a bug in this one that's a known issue for Altair. It works the first time and then stops. See the [bug docs](https://github.com/altair-viz/altair/issues/1428) for more information. 

In [34]:
#3.3.1
selectOrigin=alt.selection_single(
    bind=alt.binding_checkbox(name="hide color"),
)
hp_mpg.add_selection(
    selectOrigin
).encode(
    color=alt.condition(selectOrigin,alt.Color("Origin:N"),alt.value('gray'))
)

alt.Chart(...)

### 3.4 Slider

A really useful tool for quantitative variables is the slider. We need to know the min/max values, but otherwise it works much like the other widgets.

In [35]:
#3.4.1 
#use pandas to get the min/max

horsepower_min = cars["Horsepower"].min()
horsepower_max = cars["Horsepower"].max()
print(horsepower_min,horsepower_max)

46.0 230.0


In [38]:
#3.4.2 show both slider in transform and in condition 
#init slider 
slider=alt.binding_range(
    min=horsepower_min,
    max=horsepower_max,
    step=1,
    name="cutoff"
)

#init selection 
selector=alt.selection_single(
    bind=slider,#bind the slider
    fields=["cutoff"],
    init={"cutoff":horsepower_max}
)
sizeCondition=alt.condition(
    alt.datum.Horsepower<selector.cutoff/2,
    alt.SizeValue(300),
    alt.SizeValue(10)
)
hp_mpg.encode(
    size=sizeCondition,
    tooltip=["Name:N"]
).add_selection(
    selector
).transform_filter(
    alt.datum.Horsepower <selector.cutoff
).interactive()

alt.Chart(...)

## 4. Multiple selections and charts

You can always link selections between charts. So actions in one widget can impact multiple visualizations (or changes in one visualization will change the other... this is sometimes called cross-filtering)

In [39]:
#4.1 share selection/input 

# make a slider 
slider=alt.binding_range(
    min=horsepower_min,
    max=horsepower_max,
    step=1,
    name="cutoff"
    )

#init selection 
selector = alt.selection_single(
    bind=slider,
    fields= ["cutoff"],
    init={"cutoff":horsepower_max}
    )

# make the first chart. The size of the point
# will change based on the slider
c1=hp_mpg.encode(
    size=alt.condition(
        alt.datum.Horsepower<selector.cutoff,
        alt.SizeValue(300),alt.SizeValue(10)
    )
    ).add_selection(
        selector
    )

# the second chart has different data (actually, it's the same, just rotated)
c2=alt.Chart(cars).mark_circle(size=80,opacity=0.5).encode(
    y='Horsepower:Q',
    x='Miles_per_Gallon:Q',
    color="Origin",
    size=alt.condition(
        alt.datum.Horsepower<selector.cutoff,
        alt.SizeValue(100),alt.SizeValue(10)
    )
).add_selection(
        selector
        )
c1|c2



alt.HConcatChart(...)

In [41]:
#4.2 multiple charts,  multiple selection, and multiple conditions 

# let's put everything together. Cross filtering

# first, let's make a radio button based on number of cylinders
# first grab the initial options 
# bug: why do we need map?
# Pandas dataframe stores int as int64, which cannot be converted into json.
# Threfore, we need to convert the option list into a list of float. 
# Note: do not use pandas.to_numeric(), which will produce an incompatible format as well.   
# cy_options=list(cars["Cylinders"].unique())
cy_options=list(map(lambda x:float(x),cy_options))
cy_options.sort()

cy_radio=alt.binding_radio(
    options=cy_options
    )
radio_selector = alt.selection_single(
    name="cylinder",
    fields=["Cylinders"],
    bind=cy_radio,
    init={"Cylinders":4}
    )

# selected items should stay the original color, otherwise orangeg
colorCondition=alt.condition(radio_selector,"Cylinders",alt.value("orange"))

# make a selection based on year 
yearSelector = alt.selection_interval()

sizeCondition=alt.condition(
    yearSelector,
    alt.SizeValue(60),
    alt.SizeValue(10)
    )

# first chart, colors bound to radio
# size will be bound to the year selection
c1=alt.Chart(cars).mark_point().encode(
    y="Miles_per_Gallon:Q",
    x="Cylinders:O"
).add_selection(
        radio_selector,
    ).encode(
        color=colorCondition,
         size=sizeCondition
    )

# second chart, colors bound to radio
# size will be bound to the year selection
c2=alt.Chart(cars).mark_point().encode(
    y="Acceleration:Q",
    x="Origin:O",
    color=colorCondition,
    size=sizeCondition
)

# third chart. We going to add the selection (radio and year)
# to this one. The other two charts didn't have year.
c3=alt.Chart(cars,width=1600,height=900).mark_circle().encode(
    y="Acceleration:Q",
    x="Year:T",
    color=colorCondition,
    size=sizeCondition
).add_selection(
    yearSelector,
    radio_selector
)

# plot them all together
(c1|c2|c3).resolve_scale(y="shared")#.properties(width=1600,height=900)

alt.HConcatChart(...)